In [1]:
import tensorflow as tf
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from skimage import util
from math import exp, log
from tensorflow.python.keras.engine.base_layer import Layer
from matplotlib import rc
rc('mathtext', default='regular')

In [2]:
class MNISTLoader():
    """
    Dataset loader class
    
    """
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    
    def get_batch_test(self):
    
        return self.test_data, self.test_label
    
    def get_local_noise(self, noise_num=400, batch_size=10000):
        """
        Self-defined Gaussian-distributed noise: Generate noise_num # of gassaion noise knowing location and intensity
        
        """
        process_image = np.copy(self.test_data[:batch_size])
        W_x = intitial_W_x(process_image)
        uncer_mass = np.zeros(W_x[:,0,0,:,0].shape) + 0.01
        base_rate = np.zeros(W_x[:,0,0,:,0].shape) + 0.5
        image_size = process_image.shape[1]
        index_noise = np.random.randint(0, image_size*image_size, noise_num)
        
        for i in index_noise:
            noise_value = np.random.randn(batch_size,1) * 0.4
            feature_value = process_image[:,int(i/image_size),int(i%image_size),:]
            process_image[:,int(i/image_size),int(i%image_size),:] = feature_value + noise_value
            belief_mass = W_x[:,int(i/image_size),int(i%image_size),:,0] * (1-noise_value)
            disbelief_mass = 1 - belief_mass - uncer_mass
            W_x[:,int(i/image_size),int(i%image_size),:] = np.moveaxis(np.array([belief_mass,disbelief_mass,uncer_mass,base_rate]), 0, -1)
            
        return tf.clip_by_value(process_image, 0, 1), self.test_label[:batch_size], W_x, index_noise

    def get_noise(self, noise_mode):
        process_image = np.copy(self.test_data)
        noise_gs_img = util.random_noise(process_image,mode=noise_mode,var=0.2)
        return noise_gs_img, self.test_label
    
data_loader = MNISTLoader()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
def intitial_opinion_trust(X):
    """
    Initialize input opinion for caseI.
    
    """
    W_dis = np.zeros(X.shape).astype(np.float32)
    W_base = W_dis + 0.5
    W_x = np.array([X, W_dis, 1 - X, W_base]).astype(np.float32)
    x_trust = (W_x[0] + W_x[2] * 0.5)
    W_x = np.moveaxis(W_x, 0, -1)
    
    return W_x, x_trust

In [4]:
def intitial_W_x_location(X, index):
    """
    Initialize input opinion for caseII&III.
    
    """
    W_x = intitial_W_x(X)
    uncer_mass = np.zeros(W_x[:,0,0,:,0].shape) + 0.01
    base_rate = np.zeros(W_x[:,0,0,:,0].shape) + 0.5
    image_size = X.shape[1]
    for i in index:
        belief_mass = W_x[:,int(i/image_size),int(i%image_size),:,0]
        disbelief_mass = 1 - belief_mass - uncer_mass
        W_x[:,int(i/image_size),int(i%image_size),:] = np.moveaxis(np.array([belief_mass,disbelief_mass,uncer_mass,base_rate]), 0, -1)

    return W_x

In [5]:
def padding(opinion, kernel_size):
    """
    Trust map padding with maximum unblief mass on the padding cell. 
    
    """
    image_size = opinion.shape[1]
    p = int((kernel_size - 1)/2)
    padding_size = int(image_size + 2 * p)
    opinion_pad = np.zeros((int(opinion.shape[0]),padding_size,padding_size,int(opinion.shape[-2]),int(opinion.shape[-1])))
    opinion_pad[:,:,:,:] = np.array([0.0, 0.99, 0.01, 0.5])
    opinion_pad[:,p:p+image_size,p:p+image_size,:,:] = opinion
    return opinion_pad

In [6]:
def multi(W_x, W_y): 
    """
    Multi-sources multiplication Operator 

    """
    W_x = W_x.astype(np.float64) 
    W_y = W_y.astype(np.float64)    
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return W_b,W_d,W_u,W_a

In [7]:
def conv_single_step_trust(W_x, W_w, W_b): 
    """
    Compute the output opinion of one convolutional kernel window.
    inputs:
    W_x - input opinion (batch size, kernel size, kernel size, channel, 4)
    W_w - weight opinion (kernel size, kernel size, channel, filter number, 4)
    W_b - bias opinion (filter number, 4)
    
    """    
    W_x = W_x.astype(np.float64)   
    W_w = W_w.numpy().astype(np.float64) 
    W_b = W_b.numpy().astype(np.float64) 
    
    filter_number = W_b.shape[0]
    batch_size = W_x.shape[0]
    fusion_result = []
    W_x_expand = np.tile(np.expand_dims(W_x, axis=(4)), [1,1,1,1,filter_number,1])
    W_w_expand = np.tile(np.expand_dims(W_w, axis=(0)), [batch_size,1,1,1,1,1])
    W_b_expand = np.tile(np.expand_dims(W_b, axis=(0)), [batch_size,1,1])
    W_wx = multi(np.transpose(W_x_expand),np.transpose(W_w_expand)) # (4, 32, 3, 5, 5, 50)
    fusion_result = avg_fusion(np.asarray(W_wx), np.transpose(W_b_expand))
    
    return np.transpose(fusion_result[0]),np.transpose(fusion_result[1])

def avg_fusion(W_wx, W_b):
    """
    Multi-sources average opinion operator.
    W_wx - opinion of input mutiplied with weight opinion (4, filter number, channel, kernel size, kernel size, batch size)
    W_b - bias opinion (4, filter number, batch size)

    """
    
    W_wx = np.reshape(W_wx, (W_wx.shape[0],W_wx.shape[1], 
                             W_wx.shape[2]*W_wx.shape[3]*W_wx.shape[4], W_wx.shape[5])).astype(np.float64) # (4, 32, 75, 50)
    W_b = W_b.astype(np.float64)
    
    n_filter = W_b.shape[1]
    batch_size = W_wx.shape[-1]
    num_para = W_wx.shape[2]
    b_wx, u_wx, a_wx = W_wx[0], W_wx[2], W_wx[3]
    b_b, u_b, a_b = W_b[0], W_b[2], W_b[3]

    u_combine = np.concatenate((u_wx, np.reshape(u_b,(u_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    b_combine = np.concatenate((b_wx, np.reshape(b_b,(b_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    u_combine_recip = (np.zeros(u_combine.shape)+1)/u_combine
    
    numerator = np.sum(b_combine * u_combine_recip, axis = 1) # (32, 50)
    denominator = np.sum(u_combine_recip, axis = (1))
    
    b_fusion = numerator / denominator
    u_fusion = (num_para+1) / denominator
    a_fusion = (np.sum(a_wx, axis=(1)) + a_b) / (num_para+1)
    
    return np.array([b_fusion, 1-b_fusion-u_fusion, u_fusion, a_fusion]).astype(np.float32), (b_fusion + u_fusion * a_fusion).astype(np.float32)

In [8]:
def conv_forward(A_prev, W_w, W_b):
    """
    Forward convolutional opinion calculation.
    A_prev - feature map from last layer (batch_size, H, W, channel) 
    W_w - opinion of weight (kernel_size, kernel_size, 3, filter_num, 4)
    W_b - opinion of bias (filter_num, 4)

    """
    (batch_size, n_H_prev, n_W_prev, _, _) = A_prev.shape
    ( f , f , _, n_C, _) = W_w.shape
    stride = 1
    pad = 1
    A_prev_pad = padding(A_prev,f)

    n_H = int(( n_H_prev - f + 2 * pad )/ stride) + 1
    n_W = int(( n_W_prev - f + 2 * pad )/ stride) + 1
 
    Z = np.zeros((batch_size, n_H, n_W, n_C, 4)) 
    Z_trust = np.zeros((batch_size, n_H, n_W, n_C))

    for h in range(n_H):                       
        for w in range(n_W):                              
            vert_start = h * stride        
            vert_end = vert_start + f       
            horiz_start = w * stride        
            horiz_end = horiz_start + f     
            a_slice_prev = A_prev_pad[:,vert_start:vert_end,horiz_start:horiz_end,:,:]   
            opinion, trust = conv_single_step_trust(a_slice_prev,W_w,W_b)   
            Z[:,h,w,:,:] = opinion
            Z_trust[:,h,w,:] = trust

    return Z, Z_trust

In [9]:
def get_update_matrix(grads_w, grads_b, rs):
    """
    Update the opinion of weight and bias during training process.
    grads_w - gradient evidence matrix of weight
    grads_b - gradient evidence matrix of bias
    rs - positive evidence plus negative evidence
    
    """
    
    W_w = np.array([grads_w/(rs+2),(rs-grads_w)/(rs+2),np.full(grads_w.shape, 2/(rs+2)),
                    np.full(grads_w.shape, 0.5)]).astype(np.float32)
    W_b = np.array([grads_b/(rs+2), (rs-grads_b)/(rs+2), np.full(grads_b.shape, 2/(rs+2)), 
                    np.full(grads_b.shape, 0.5)]).astype(np.float32)
    W_w = np.moveaxis(W_w, 0, -1)
    
    return W_w, W_b.swapaxes(0,1)

In [10]:
def evidence_collect(y, y_pred):
    """
    Collect positive and negative evidence during the training process.

    """
    r = 0
    s = 0
    r_list = [0]*10
    s_list = [0]*10
    
    for j in range(len(y_pred)):
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.8:
                    r_list[i]+=1
                    r+=1
                else:
                    s+=1
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.2:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5])

    
    return [r/(r+s+2), s/(r+s+2), 2/(r+s+2), 0.5], y_N_op

In [11]:
class MyLayerMaxTrust(tf.keras.layers.Layer):
    """
    Max-trust fuction layer
    A new pooling layer function based on trustworthiness values instead of feature values. 

    """
    def __init__(self):
        super(MyLayerMaxTrust, self).__init__()


    def call(self, x, opinion, trust):
        """
        x - feature map from last layer (batch, H, W, C)
        opinion - corresponding opinion (batch, H, W, C, 4)
        trust - corresponding trust (batch, H, W, C)

        """
        trust_mul = np.zeros(x.shape)
        image_size = x.shape[1]
        opinion_out = np.zeros((opinion.shape[0],int(opinion.shape[1]/2), int(opinion.shape[2]/2),opinion.shape[3], 4)).astype(np.float32)
        for i in range(trust.shape[0]):
            for k in range(trust.shape[-1]):
                input_max = trust[i,:,:,k].reshape((1,trust.shape[1],trust.shape[2],1))
                _, argmax = tf.nn.max_pool_with_argmax(input = input_max, ksize = [1, 2, 2, 1],
                                                    strides = [1, 2, 2, 1], padding = 'VALID')
                argmax_1d = argmax.numpy().flatten()
                for j in range(len(argmax_1d)):
                    trust_mul[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k] = 1
                    opinion_out[i,int(j/int(image_size/2)), int(j%int(image_size/2)),k,:] = opinion[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k,:] 

        x = x * trust_mul

        return x, opinion_out

In [12]:
class MyLayerCONV(tf.keras.layers.Layer):
    """
    Conv opinion computation layer
    
    """
    def __init__(self):
        super(MyLayerCONV, self).__init__()


    def call(self, input1, input2, input3):
        
        Z, Z_trust = conv_forward(input1, input2, input3)
           
        return Z.astype(np.float32), Z_trust.astype(np.float32)

In [13]:
class MyLayerMaxPool(tf.keras.layers.Layer):
    """
    Original max-pooling with trust framework
     
    """
    def __init__(self):
        super(MyLayerMaxPool, self).__init__()


    def call(self, x, opinion):
        """
        x-(50,14,14,8)
        
        """
        image_size = x.shape[1]
        opinion_out = np.zeros((opinion.shape[0],int(opinion.shape[1]/2), 
                                int(opinion.shape[2]/2),opinion.shape[3], 4)).astype(np.float32)

        for i in range(x.shape[0]):
            for k in range(x.shape[-1]):
                input_max = x[i,:,:,k].numpy().reshape((1,x.shape[1],x.shape[2],1))
                _, argmax = tf.nn.max_pool_with_argmax(input = input_max, ksize = [1, 2, 2, 1],
                                                    strides = [1, 2, 2, 1], padding = 'VALID')
                argmax_1d = argmax.numpy().flatten()
                for j in range(len(argmax_1d)):
                    opinion_out[i,int(j/int(image_size/2)), int(j%int(image_size/2)),k,:] = opinion[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k,:] 
        return opinion_out

In [14]:
filter_num1 = 4
filter_num2 = 8
filter_num3 = 8
kernelsize = 3

In [15]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.convopinion1 = MyLayerCONV()
        self.convopinion2 = MyLayerCONV()
        self.convopinion3 = MyLayerCONV()

        self.conv1 = tf.keras.layers.Conv2D(
            filters=filter_num1,             
            kernel_size=[kernelsize, kernelsize],
            padding='same',
            activation=tf.nn.relu   
        )
        self.conv2 = tf.keras.layers.Conv2D(
            filters=filter_num2,           
            kernel_size=[kernelsize, kernelsize],    
            padding='same',
            activation=tf.nn.relu   
        )
        self.conv3 = tf.keras.layers.Conv2D(
            filters=filter_num3,            
            kernel_size=[kernelsize, kernelsize],     
            padding='same',         
            activation=tf.nn.relu   
        )

        self.maxtrust1 = MyLayerMaxTrust()
        self.maxtrust2 = MyLayerMaxTrust()
        self.maxfeature1 = MyLayerMaxPool()
        self.maxfeature2 = MyLayerMaxPool()
        
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
       
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)

    def call(self, inputs):
        """
        x - 50,32,32,3
        W_x - 50,32,32,3,4
        
        
        """
        X, W_x, W_w1, W_b1, W_w2, W_b2 = inputs
 
        opinion1, trust1 = self.convopinion1(W_x, W_w1, W_b1)
        x = self.conv1(X)   
        x, pooling_opinion1 = self.maxtrust1(x, opinion1, trust1)
        x = self.pool1(x) 
        opinion2, _ = self.convopinion2(pooling_opinion1, W_w2, W_b2)
        x = self.conv2(x)
        pooling_opinion2 = self.maxfeature2(x, opinion2)
        x = self.pool2(x)    
        x = self.flatten(x) 
        x = self.dense1(x)    
        output = self.dense2(x)                    
        return output,opinion1,opinion2,pooling_opinion2
    
model = CNN()

In [16]:
num_epochs = 2
batch_size = 50
threshold1 = 3.0
threshold2 = 5.0
optimizer = tf.keras.optimizers.Adam()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [17]:
# initial parameters opinion

w_initial1 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)] 
                         for _ in range(1)] for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)
b_initial1 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)]).astype(np.float32)

w_initial2 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num2)] 
                         for _ in range(filter_num1)] for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)
b_initial2 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num2)]).astype(np.float32)

In [18]:
train_acc = []
train_loss = []
opinion_convlist = []
y_update_wb = []
y_N_opinion = []

In [ ]:
# training process
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
W_w1 = w_initial1
W_b1 = b_initial1
W_w2 = w_initial2
W_b2 = b_initial2

grads_list_w1 = np.zeros((kernelsize,kernelsize,1,filter_num1))
grads_list_b1 = np.zeros((filter_num1,))
grads_list_w2 = np.zeros((kernelsize,kernelsize,filter_num1,filter_num2))
grads_list_b2 = np.zeros((filter_num2,))

import time
start = time.time()

for batch_index in range(num_batches):
    print('# of batch:',batch_index)
    X, y = data_loader.get_batch(batch_size)  
    W_x ,_ = intitial_opinion_trust(X)
    with tf.GradientTape() as tape:
        y_pred,opinion_conv1,opinion_conv2,opinion_conv = model([X, W_x, W_w1, W_b1, W_w2, W_b2])
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    
    sparse_categorical_accuracy.update_state(y_true=y, y_pred=y_pred)
    opinion_convlist.append(np.reshape(opinion_conv, (opinion_conv.shape[0],int(opinion_conv.shape[1]*opinion_conv.shape[2]*opinion_conv.shape[3]),4)))
    train_loss.append(tf.reduce_mean(loss).numpy())
    train_acc.append(sparse_categorical_accuracy.result().numpy())
    print("train accuracy:",sparse_categorical_accuracy.result().numpy())
   
# update W_w, W_b
    grads_list_w1 = grads_list_w1 + tf.where(abs(grads[0])<threshold1, 1, 0)  
    grads_list_b1 = grads_list_b1 + tf.where(abs(grads[1])<threshold1, 1, 0)  
    grads_list_w2 = grads_list_w2 + tf.where(abs(grads[2])<threshold2, 1, 0)  
    grads_list_b2 = grads_list_b2 + tf.where(abs(grads[3])<threshold2, 1, 0) 
    W_w1, W_b1 = get_update_matrix(grads_list_w1, grads_list_b1, batch_index+1)
    W_w2, W_b2 = get_update_matrix(grads_list_w2, grads_list_b2, batch_index+1)

    y_N_update, y_N_op = evidence_collect(y, y_pred)
    y_N_opinion.append(y_N_op)
    y_update_wb.append(y_N_update)
    print('One batch end:',(time.time()-start))

In [ ]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [ ]:
# model.save_weights('my_model_max_trust_MINIST')
# save_variable(W_w1, 'MINST_weight_opinion1')
# save_variable(W_w2, 'MINST_weight_opinion2')
# save_variable(W_b1, 'MINIST_bias_opinion1')
# save_variable(W_b2, 'MINIST_bias_opinion2')

In [ ]:
model.load_weights('my_model_max_trust_MINIST')
W_w1 = load_variavle('max_trust_MINST_weight_opinion1')
W_w2 = load_variavle('max_trust_MINST_weight_opinion2')
W_b1 = load_variavle('max_trust_MINIST_bias_opinion1')
W_b2 = load_variavle('max_trust_MINIST_bias_opinion2')

In [ ]:
# X_test, y_test= data_loader.get_batch_test()
# W_x ,_ = intitial_opinion_trust(X_test)
# y_pred_test, _,_,_ = model([X_test, W_x, W_w1, W_b1, W_w2, W_b2])
# sparse_categorical_accuracy.reset_states()
# sparse_categorical_accuracy.update_state(y_true=y_test, y_pred=y_pred_test)
# print("test accuracy:",sparse_categorical_accuracy.result().numpy())

In [ ]:
# Evaluation of noisy dataset
X_test_noise, y_test_noise = data_loader.get_noise('gaussian')
noise_opinion,_ = intitial_opinion_trust(X_test_noise)
y_pred_test_noise,opinion_conv1,opinion_conv2,opinion_conv = model([X_test_noise, noise_opinion, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_noise, y_pred=y_pred_test_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

In [ ]:
# Evaluation of three cases
X_test_local_noise, y_test_local_noise, case1_opinion, noise_index = data_loader.get_local_noise()
case2_opinion = intitial_W_x_location(X_test_local_noise, noise_index)
case3_opinion = intitial_W_x(X_test_local_noise)
y_pred_test_local_noise,_,_,opinion_noise = model([X_test_local_noise, case1_opinion, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_local_noise, y_pred=y_pred_test_local_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

In [ ]:
def get_NN_trust(opinion_last_layer, true_label, pre_label, y_update_wb):
    # compute dense opinion
    opinion_dense = np.average(np.array(opinion_last_layer), axis=0)
    opinion_dense = np.reshape(opinion_dense, (int(opinion_dense.shape[0]*opinion_dense.shape[1]*opinion_dense.shape[2]),4))
    y_true_op = [1.0, 0.0, 0.0, 0.5]
    W_y_update = evidence_collect_test(true_label, pre_label)
    
    # compute Backward opinion of neuron W_N_Y  
    W_N_Y=[]
    for j in W_y_update:
        W_N_Y.append(multi(j,y_true_op)) # change when add flaw in label
        
    W_w_dense = y_update_wb[-1]
    W_b_dense = y_update_wb[-1]

    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(opinion_dense.shape[0]):
        W_xw.append(multi(W_w_dense,opinion_dense[j])) # (513, 4)
    dense_out = fusion(np.array(W_xw))
    #     print('Underflow or not: ',np.isnan(np.min(np.array(dense1_out_list))))

    # last layer
    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(64):
        W_xw.append(multi(W_w_dense,dense_out))
    W_NN = fusion(np.array(W_xw))
    
    # compute last layer output opinion and trust
    W_XY_one = []
    for j in range(10):
        W_XY_one.append(fusion_2(W_NN,W_N_Y[j]))
    W_NN = fusion(np.array(W_XY_one))
    W_trust = W_NN[0]+W_NN[2]*W_NN[3]
    
    return W_trust, W_NN

In [ ]:
# forward opinion prop
def mul_scale(W_x):
    mul_u = 1.0
    for i in range(len(W_x)):
        mul_u = mul_u * W_x[i]
        if mul_u <= 1e-24: 
            mul_u = mul_u*1e25
    return mul_u

def fusion(W_x): # W_x array

    deno = 0.0
    mole = 0.0
    full_multi = mul_scale(W_x[:,2])
    for i in range(len(W_x[:,2])):
        deno = deno + full_multi/W_x[:,2][i]
        mole = mole + (full_multi/W_x[:,2][i])*W_x[:,0][i]
    W_b = mole/deno  #  change
    W_u = (len(W_x)*full_multi)/deno   # change
    W_a = sum(W_x[:,3])/len(W_x)  

#     W_b = sum(1/len(W_x)*W_x[:,0])
#     W_u = 0.0
#     W_a = sum(1/len(W_x)*W_x[:,3])
        
    return [W_b,1-W_b-W_u,W_u,W_a]

def fusion_2(W_x, W_y):
    if W_x[2]!=0 or W_y[2]!=0:
        W_b = (W_x[0]*W_y[2]+W_y[0]*W_x[2])/(W_x[2]+W_y[2])
        W_u = 2*W_x[2]*W_y[2]/(W_x[2]+W_y[2])
        W_a = (W_x[3]+W_y[3])/2  
    elif W_x[2]==0 and W_y[2]==0:
        W_b = 0.5*W_x[0]+0.5*W_y[0]
        W_u = 0
        W_a = 0.5*W_x[3]+0.5*W_y[3]
    return [W_b,1-W_b-W_u,W_u,W_a]

def multi(W_x, W_y): # 
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return [W_b,W_d,W_u,W_a]


def evidence_collect_test(y, y_pred):
    r_list = [0]*10
    s_list = [0]*10
    index = np.random.randint(0,y.shape[0], 5000)
    
    for j in index:
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.9:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.1:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append(np.array([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5]))

    
    return y_N_op

In [ ]:
# compute Backward opinion of neuron W_N_Y  
W_N_Y_noise=[]
for j in W_y_update:
    W_N_Y_noise.append(multi(j,y_true_op)) # change when add flaw in label

In [ ]:
opinion_dense_noise = np.average(np.array(opinion_mid), axis=0)
opinion_dense_noise = np.reshape(opinion_dense_noise, (int(opinion_dense_noise.shape[0]*opinion_dense_noise.shape[1]*opinion_dense_noise.shape[2]),4))

In [ ]:
W_w_dense = y_update_wb[-1]
W_b_dense = y_update_wb[-1]
input_dense = opinion_dense_noise

W_xw=[]
W_xw.append(W_b_dense)
for j in range(opinion_dense_noise.shape[0]):
    W_xw.append(multi(W_w_dense,input_dense[j]))
dense_out_noise = fusion(np.array(W_xw))
#     print('Underflow or not: ',np.isnan(np.min(np.array(dense1_out_list))))

# last layer
W_xw=[]
W_xw.append(W_b_dense)
for j in range(64):
    W_xw.append(multi(W_w_dense,dense_out_noise))
W_NN_noise = fusion(np.array(W_xw))

In [ ]:
# compute last layer output opinion and trust
W_XY_one = []
for j in range(10):
    W_XY_one.append(fusion_2(W_NN_noise,W_N_Y_noise[j]))

W_NN = fusion(np.array(W_XY_one))
W_trust = W_NN[0]+W_NN[2]*W_NN[3]

In [ ]:
entropy = -1 * (W_trust * np.log2(W_trust) + (1-W_trust) * np.log2(1-W_trust))
entropy